# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [314]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

data = pd.read_csv("jeopardy.csv")
data.head() #looking at the first 5 rows of the data to see the columns.

# data.info() #looking at the data types present in the table. It appears there are 2 questions which do not have answers.
#it also appears the column names have additional white space infront.

data.columns = data.columns.str.strip()

# data.info() #confirming the columns names have had additional whitespace removed.

data = data.rename(columns = {"Air Date": "Air_date", "Show Number":"Show_number"}) #renaming air date and show number columns using function methods.

# data.info() confirming the rename has worked

data = data.loc[data.Answer.notnull()] #cleaning the data to remove rows where an answer is not given.

data.info() #confirming that the amount of not null data in each column is the same

<class 'pandas.core.frame.DataFrame'>
Index: 216928 entries, 0 to 216929
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Show_number  216928 non-null  int64 
 1   Air_date     216928 non-null  object
 2   Round        216928 non-null  object
 3   Category     216928 non-null  object
 4   Value        216928 non-null  object
 5   Question     216928 non-null  object
 6   Answer       216928 non-null  object
dtypes: int64(1), object(6)
memory usage: 13.2+ MB


2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [316]:
def find_question_with_words(word_list, df):
    filtered_frame = df #initializes a temporary filtered frame from the given data frame
    for word in word_list: #loops through each word in the word list and filters out the questions which contain that word. 
        filtered_frame = filtered_frame.loc[filtered_frame.Question.str.contains(word) == True]
    return filtered_frame #only questions which contains all words in the list are returned

filtered = find_question_with_words(["King", "England"], data)
filtered.info() #checking this matches with the given number

<class 'pandas.core.frame.DataFrame'>
Index: 49 entries, 4953 to 200369
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Show_number  49 non-null     int64 
 1   Air_date     49 non-null     object
 2   Round        49 non-null     object
 3   Category     49 non-null     object
 4   Value        49 non-null     object
 5   Question     49 non-null     object
 6   Answer       49 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.1+ KB


3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [368]:
def find_question_with_words(word_list, df):
    filtered_frame = df #initializes a temporary filtered frame from the given data frame
    for word in word_list: #loops through each word in the word list and filters out the questions which contain that word. 
        # filtered_frame = filtered_frame.loc[(filtered_frame.Question.str.contains(fr'\b\W?{word}s?\W?\b', case = False, regex = True) == True) | (filtered_frame.Question.str.contains(fr'\b\W?{word}\W?s?\b', case = False, regex = True) == True) | (filtered_frame.Question.str.contains(fr'\s+\W?{word}\b', case = False, regex = True) == True)] #1
        filtered_frame = filtered_frame.loc[filtered_frame.Question.str.contains(fr'\b\W?{word}\W?s?\b', case = False, regex = True) == True]
    return filtered_frame #only questions which contains all words in the list are returned

# the three regex expression intend to serve 3 purposes
# the first regex expression tries to capture when sentences start with the word as they would not contain whitespace before hand
# the side effect of this first expression is that it can incorrectly pick up words containing the word, like thanking, unsure how to avoid.
# the 2nd expression intends to pick up the words themselves inside a sentence, allowing for punctuation to preceed or proceed the word
# the 3rd expression intends to pick up when the words end the sentence, this has the unintended consequence of picking up words like kinging
# where the word is a substring in another word. Unsure how to avoid, or force the last letter of the word to be last letter of the searched string.

#2 using \b imporved the implementation of the boundry checks which solved the issue of including "viking" and "thanking"

filtered = find_question_with_words(["King", "England"], data)
filtered.info() #shows 130 matches for questions which contain the word King and England in any case or plural
print(filtered.Question)
#the filter is struggling to ignore words which contain the word as a substring such as viking and thanking. (now solved)

<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, 4953 to 216021
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Show_number  130 non-null    int64  
 1   Air_date     130 non-null    object 
 2   Round        130 non-null    object 
 3   Category     130 non-null    object 
 4   Value        130 non-null    float64
 5   Question     130 non-null    object 
 6   Answer       130 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 8.1+ KB
4953                                                                                                                                                                                                                                                                                                     Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337                                                                 

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [320]:
data.Value = data.Value.apply(lambda x: x.replace("$", "") if type(x) == str else float(x)) #replace dollar signs
data.Value = data.Value.apply(lambda x: x.replace(",", "") if type(x) == str else float(x)) #replace commas
data.Value = data.Value.apply(lambda x: 0 if x == "no value" else float(x)) #convert values to floats and replace "no value" columns with 0
# print(data.info()) #confirming data change

king_filtered = find_question_with_words(["King"], data)
king_median_value = king_filtered.Value.median()
print(king_median_value) #the median value of the questions containing king is around 600
king_mean_value = king_filtered.Value.mean()
print(king_mean_value) # the mean value of questions containing king is around 771

600.0
771.8599463504165


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [322]:
def n_unique_answers(df):
    df.Answer.apply(lambda x: x.lower().strip())
    num_unique = df.groupby("Answer").Question.count().reset_index(name = "count").sort_values(["count"], ascending = False)
    return num_unique

henry_vii = n_unique_answers(king_filtered) #shows 54 for henry viii as there is an additional question in solution code which includes a question
#which only contains king as part of buckingham, however my filter does not include this.
henry_vii.head()

,Answer,count
1483,Henry VIII,54
2837,Solomon,35
1987,Louis XIV,31
2607,Richard III,31
977,David,30


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [352]:
import random

data.reset_index()
def select_random_question(data):
    selection = data.iloc[random.randint(0,data.Question.count())] #selects a random question using iloc and random number generated from 0 to the number of questions in the dataframe
    question = selection.Question #stores the question in a variable
    answer = selection.Answer #stores the answer in a variable
    print(question) #prints the question
    player_answer = input("Please provide an answer: ") #input for the player to provide an answer
    if player_answer.lower().strip() in answer.lower().strip(): #checks if the answer matches or partially matches the correct one
        print(f"That is correct! The answer was {answer}") #statement if answer is correct
    else:
        print(f"That is incorrect :(, the correct answer is {answer}") #statement if answer is incorrect
    

select_random_question(data)

This name properly refers to only the great hour bell at the northern end of London's Houses of Parliament


Please provide an answer:  big ben


That is correct! The answer was Big Ben


## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.